In [1]:
import scipy.io as sio
from PIL import Image
import cv2
import numpy as np
import os
import pickle

ANNOTATION_PATH = "..\\data\\AHP\\AHP\\train\\Annotations\\"
IMAGE_PATH = "..\\data\\AHP\\AHP\\train\\JPEGImages\\"
TRAIN_MAPPING = "data\\AHP\\train_annotations.pkl"
TEST_MAPPING = "data\\AHP\\test_annotations.pkl"
IMG_DIR = "data\\AHP\\AHP\\train\\Processed_Images"
GT_DIR = "data\\AHP\\AHP\\train\\Processed_Annotations"

In [22]:
''' Get average proportion of pixels in each quadrant that are human '''

imgs = os.listdir(ANNOTATION_PATH)

proportions_total = {"00": 0, "01": 0, "02": 0, "10": 0, "11": 0, "12":0, "20": 0, "21": 0, "22":0}

for img in imgs:
    sample = cv2.cvtColor(cv2.imread(ANNOTATION_PATH + img), cv2.COLOR_BGR2RGB)[:,:,0]
    height, width = sample.shape
    sample[sample > 1] = 1


    rows_step = int(height / 3)
    cols_step = int(width / 3)

    for i in range(3):
        for j in range(3):
            subarray = sample[i*rows_step:(i+1)*rows_step, j*cols_step:(j+1)*cols_step]
            # print(subarray.shape)
            w, h = subarray.shape
            total_pixels = w * h
            total_human_pixels = np.sum(subarray)


            proportions_total[str(i) + str(j)] += (total_human_pixels / total_pixels)

x = {q: p / len(imgs) for q, p in proportions_total.items()}


TypeError: ufunc 'exp' not supported for the input types, and the inputs could not be safely coerced to any supported types according to the casting rule ''safe''

In [9]:
'''Augment Dataset with translated images'''

import matplotlib.pyplot as plt

imgs = os.listdir(ANNOTATION_PATH)

proportions_total = {"00": 0, "01": 0, "02": 0, "10": 0, "11": 0, "12":0, "20": 0, "21": 0, "22":0}



for num, img in enumerate(imgs):
    annot = cv2.imread(ANNOTATION_PATH + img)
    image = cv2.imread(IMAGE_PATH + img.split(".")[0] + ".jpg")
    height, width, channels= annot.shape

  

    if num < 10000:
        # Creating a translation matrix
        translation_matrix = np.float32([ [1,0,int(width/3)], [0,1,0] ])
        # Image translation
        img_translation = cv2.warpAffine(image, translation_matrix, (width,height))
        annot_translation = cv2.warpAffine(annot, translation_matrix, (width,height))

        cv2.imwrite(IMAGE_PATH + img.split(".")[0] + "_righttranslated.jpg", img_translation, )
        cv2.imwrite(ANNOTATION_PATH + img.split(".")[0] + "_righttranslated.png", annot_translation)

    if num > 10000:
        # Creating a translation matrix
        translation_matrix = np.float32([ [1,0,-int(width/3)], [0,1,0] ])
        # Image translation
        img_translation = cv2.warpAffine(image, translation_matrix, (width,height))
        annot_translation = cv2.warpAffine(annot, translation_matrix, (width,height))

        cv2.imwrite(IMAGE_PATH + img.split(".")[0] + "_lefttranslated.jpg", img_translation)
        cv2.imwrite(ANNOTATION_PATH + img.split(".")[0] + "_lefttranslated.png", annot_translation)

    if num > 20000: break